# Выбор локации для скважины

Для исследования предоставлены три датасета компании «ГлавРосГосНефть», в них хранятся данные геолоразведки нефтеных скважин трёх регионов.

В каждом датасете 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыл, проанализировать возможную прибыль и риски техникой Bootstrap.

Согласно документации к данным:
* `id` — уникальный идентификатор скважины;
* `f0, f1, f2` — три признака точек;  
* `product` — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

In [3]:
import pandas as pd
import numpy as np
#import pandas_profiling

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy import stats as st

Загрузим и проверим данные на налие пропусков и несоответствие типов данных.

In [4]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

В данных нет пропусков и значения соответсвуют типам данных. Удалим столбец id, для обучения модели он не требуется.

In [5]:
data_0 = data_0.drop('id', axis = 1)
data_1 = data_1.drop('id', axis = 1)
data_2 = data_2.drop('id', axis = 1)

Разделим данные на выборки.

In [6]:
features_0 = data_0.drop(['product'], axis = 1)
target_0 = data_0['product']

features_1 = data_1.drop(['product'], axis = 1)
target_1 = data_1['product']

features_2 = data_2.drop(['product'], axis = 1)
target_2 = data_2['product']

#features_train - обучающая выборка
#features_valid - валидационая выборка

features_train_0, features_valid_0, target_train_0, target_valid_0 = \
train_test_split(features_0, target_0, test_size = 0.25, random_state = 12345)

features_train_1, features_valid_1, target_train_1, target_valid_1 = \
train_test_split(features_1, target_1, test_size = 0.25, random_state = 12345)

features_train_2, features_valid_2, target_train_2, target_valid_2 = \
train_test_split(features_2, target_2, test_size = 0.25, random_state = 12345)

Проверим размеры выборок.

In [7]:
print('features_train_0:')
print(features_train_0.shape)
print(target_train_0.shape)

print('features_valid_0:')
print(features_valid_0.shape)
print(target_valid_0.shape)
print()

print('features_train_1:')
print(features_train_1.shape)
print(target_train_1.shape)

print('features_valid_1:')
print(features_valid_1.shape)
print(target_valid_1.shape)
print()

print('features_train_2:')
print(features_train_2.shape)
print(target_train_2.shape)

print('features_valid_2:')
print(features_valid_2.shape)
print(target_valid_2.shape)
print()

features_train_0:
(75000, 3)
(75000,)
features_valid_0:
(25000, 3)
(25000,)

features_train_1:
(75000, 3)
(75000,)
features_valid_1:
(25000, 3)
(25000,)

features_train_2:
(75000, 3)
(75000,)
features_valid_2:
(25000, 3)
(25000,)



Приведём численные признаки к одному масштабу.

In [8]:
numeric = ['f0', 'f1', 'f2']

scaler_0 = StandardScaler()
scaler_0.fit(features_train_0[numeric])
features_train_0[numeric] = scaler_0.transform(features_train_0[numeric])
features_valid_0[numeric] = scaler_0.transform(features_valid_0[numeric])

scaler_1 = StandardScaler()
scaler_1.fit(features_train_1[numeric])
features_train_1[numeric] = scaler_1.transform(features_train_1[numeric])
features_valid_1[numeric] = scaler_1.transform(features_valid_1[numeric])

scaler_2 = StandardScaler()
scaler_2.fit(features_train_2[numeric])
features_train_2[numeric] = scaler_2.transform(features_train_2[numeric])
features_valid_2[numeric] = scaler_2.transform(features_valid_2[numeric])

print('features_train_0:')
print(features_train_0.shape)
print(features_valid_0.shape)
print()

print('features_train_1:')
print(features_train_1.shape)
print(features_valid_1.shape)
print()

print('features_train_2:')
print(features_train_2.shape)
print(features_valid_2.shape)

features_train_0:
(75000, 3)
(25000, 3)

features_train_1:
(75000, 3)
(25000, 3)

features_train_2:
(75000, 3)
(25000, 3)


/tmp/ipykernel_31/1191553689.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_train_0[numeric] = scaler_0.transform(features_train_0[numeric])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

## Обучение и проверка модели

In [9]:
def modelfit(features, target, features_valid, target_valid):
    model = LinearRegression()
    model.fit(features, target)
    prediction = model.predict(features_valid)
    print('MSE =', mean_squared_error(target_valid, prediction))
    print('RMSE =', mean_squared_error(target_valid, prediction) ** 0.5)
    print('R2 =', r2_score(target_valid, prediction))
    print('Средний запас предсказанного сырья:', prediction.mean())
    return prediction

print('Результаты обучения модели для нулевой области:')
predictions_valid_0 = modelfit(features_train_0, target_train_0, features_valid_0, target_valid_0)
print()

print('Результаты обучения модели для первой области:')
predictions_valid_1 = modelfit(features_train_1, target_train_1, features_valid_1, target_valid_1)
print()

print('Результаты обучения модели для второй области:')
predictions_valid_2 = modelfit(features_train_2, target_train_2, features_valid_2, target_valid_2)

Результаты обучения модели для нулевой области:
MSE = 1412.2129364399243
RMSE = 37.5794217150813
R2 = 0.27994321524487786
Средний запас предсказанного сырья: 92.59256778438035

Результаты обучения модели для первой области:
MSE = 0.7976263360391157
RMSE = 0.893099286775617
R2 = 0.9996233978805127
Средний запас предсказанного сырья: 68.728546895446

Результаты обучения модели для второй области:
MSE = 1602.3775813236196
RMSE = 40.02970873393434
R2 = 0.20524758386040443
Средний запас предсказанного сырья: 94.96504596800489


У нулевой модели самое наименьшее значение квадрата среднеквадратичной ошибки, что говорит о меньшем кол-ве грубых ошибок по сравнению с остальными моделями.

## Подготовка к расчёту прибыли

Создадим переменные для расчёта прибыли и функцию для подсчёта выручки.

In [10]:
predictions_valid_0_copy = pd.Series(predictions_valid_0)
predictions_valid_1_copy = pd.Series(predictions_valid_1)
predictions_valid_2_copy = pd.Series(predictions_valid_2)


predictions_valid_0_series = pd.Series(predictions_valid_0)
target_valid_0 = pd.Series(target_valid_0)

predictions_valid_1_series = pd.Series(predictions_valid_1)
target_valid_1 = pd.Series(target_valid_1)

predictions_valid_2_series = pd.Series(predictions_valid_2)
target_valid_2 = pd.Series(target_valid_2)

target_valid_0 = target_valid_0.reset_index(drop = True)
predictions_valid_0_series = predictions_valid_0_series.reset_index(drop = True)

target_valid_1 = target_valid_1.reset_index(drop = True)
predictions_valid_1_series = predictions_valid_1_series.reset_index(drop = True)

target_valid_2 = target_valid_2.reset_index(drop = True)
predictions_valid_2_series = predictions_valid_2_series.reset_index(drop = True)

BUDGET = 10_000_000_000
REVENUE = 450_000
N_DOT = 200

def profit(target, predicted):
    pred_sorted = predicted.sort_values(ascending = False)
    selected = target[pred_sorted.index][:N_DOT]
    return REVENUE * selected.sum() - BUDGET

res_0 = profit(target_valid_0, predictions_valid_0_copy)
res_1 = profit(target_valid_1, predictions_valid_1_copy)
res_2 = profit(target_valid_2, predictions_valid_2_copy)


print('Нулева область добычи:', res_0)
print('Первая область добычи:', res_1)
print('Вторая область добычи:', res_2)

Нулева область добычи: 3320826043.1398506
Первая область добычи: 2415086696.681511
Вторая область добычи: 2710349963.5998325


Наибольшая прибыль достигается в первой области.

## Расчёт прибыли и рисков 

In [11]:
%%time

state = np.random.RandomState(12345)

def boost(target, prediction):
    values = []
    for i in range(1000):
        target_subsample = target.sample(n = 500, replace = True, random_state = state)
        probs_subsample = prediction[target_subsample.index]
        values.append(profit(target_subsample, probs_subsample))
    values = pd.Series(values)
    print('Среднее:', values.mean())
    print('Интервал:', values.quantile(0.025), '-', values.quantile(0.975))
    counter = 0
    for elem in values:
        if elem < 0:
            counter += 1
        else:
            counter += 0
    print('Риск убытков:', counter / len(values) * 100, '%')

print('Нулевая область:')
boost(target_valid_0, predictions_valid_0_series)
print()

print('Первая область:')
boost(target_valid_1, predictions_valid_1_series)
print()

print('Вторая область:')
boost(target_valid_2, predictions_valid_2_series)

Нулевая область:
Среднее: 425938526.91059244
Интервал: -102090094.83793654 - 947976353.3583689
Риск убытков: 6.0 %

Первая область:
Среднее: 518259493.6973249
Интервал: 128123231.43308444 - 953612982.0669085
Риск убытков: 0.3 %

Вторая область:
Среднее: 420194005.3440501
Интервал: -115852609.16001143 - 989629939.8445739
Риск убытков: 6.2 %
CPU times: user 3.98 s, sys: 0 ns, total: 3.98 s
Wall time: 4 s


Для самой прибыльной области (первой):

Средняя прибыль максимальна из всех областей. И с вероятностью 95% прибыль может быть от 128000000 до 953000000.

## Выводы

Для исследования были предоставлены данные трёх областей по добычи нефти. В данных не было пропусков все значени корректны. Для каждой области были обучены модели линейной регрессии. Для дальнейших расчётов была выбранна нулевая модель соответственно и нулева область. Для вычисления возможной доходности скважин в области была использована техника bootstrap. В данной области возможен самый высокий доход, посравнению с остальными областями.